In [2]:
#Importar libreria para trabajar con pandas
%additional_python_modules awswrangler

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.2 
Additional python modules to be included:
awswrangler


In [152]:
#---- Librerias python ---#
import json
import datetime
import boto3
import pandas as pd
import numpy as np
import awswrangler as wr

In [153]:
#Configuracion de pandas para visualizar las filas y columnas completas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [154]:
#Definir lo paths 
path_source="s3://wo-data-origen/hotel_bookings_raw.csv"
path_target = 's3://wo-data-procesada/hotel_data/'

In [155]:
#cargar datos desde s3 usando awswrangler
df = wr.s3.read_csv(path=path_source)

### Seleccionar solo las columnas que usaremos

In [156]:
df = df[[ 'hotel', 'is_canceled', 'adults', 'children', 'babies', 'meal',
       'country', 'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled','deposit_type','market_segment',
       'reservation_status', 'reservation_status_date','arrival_date_year']]

In [157]:
#Imprimir un registro
df.head(1)

          hotel  is_canceled  adults  children  babies meal country  \
0  Resort Hotel            0       2       0.0       0   BB     PRT   

   is_repeated_guest  previous_cancellations  previous_bookings_not_canceled  \
0                  0                       0                               0   

  deposit_type market_segment reservation_status reservation_status_date  \
0   No Deposit         Direct          Check-Out                7/1/2015   

   arrival_date_year  
0               2015


### Crearemos un index por defecto y lo renombraremos con el nombre id_reservation

In [158]:
df = df.reset_index()

In [159]:
df = df.rename(columns={'index':'id_reservation'})

In [160]:
df.head(5)

   id_reservation         hotel  is_canceled  adults  children  babies meal  \
0               0  Resort Hotel            0       2       0.0       0   BB   
1               1  Resort Hotel            0       2       0.0       0   BB   
2               2  Resort Hotel            0       1       0.0       0   BB   
3               3  Resort Hotel            0       1       0.0       0   BB   
4               4  Resort Hotel            0       2       0.0       0   BB   

  country  is_repeated_guest  previous_cancellations  \
0     PRT                  0                       0   
1     PRT                  0                       0   
2     GBR                  0                       0   
3     GBR                  0                       0   
4     GBR                  0                       0   

   previous_bookings_not_canceled deposit_type market_segment  \
0                               0   No Deposit         Direct   
1                               0   No Deposit         Dir

### Reemplazar datos

In [161]:
df['hotel'] = df['hotel'].replace({"Resort Hotel": 'ResortHotel', "City Hotel": 'CityHotel'})

In [162]:
df['hotel'].unique()

array(['ResortHotel', 'CityHotel'], dtype=object)


### Filtrar la información del año mas reciente

In [163]:
#Los valores unicos de la columna arrival_date_year son
df['arrival_date_year'].unique()

array([2015, 2016, 2017])


In [164]:
df = df[df['arrival_date_year'] == 2017]

In [165]:
df['arrival_date_year'].unique()

array([2017])


In [166]:
#validamos que la cantidad de registros ha disminuido
len(df)

40687


In [167]:
df.head(3)

      id_reservation        hotel  is_canceled  adults  children  babies meal  \
9775            9775  ResortHotel            1       2       0.0       0   BB   
9776            9776  ResortHotel            1       2       0.0       0   BB   
9777            9777  ResortHotel            1       2       0.0       0   BB   

     country  is_repeated_guest  previous_cancellations  \
9775     PRT                  0                       0   
9776     AUT                  0                       0   
9777     AUT                  0                       0   

      previous_bookings_not_canceled deposit_type market_segment  \
9775                               0   No Deposit      Online TA   
9776                               0   No Deposit      Online TA   
9777                               0   No Deposit      Online TA   

     reservation_status reservation_status_date  arrival_date_year  
9775           Canceled              10/20/2016               2017  
9776           Canceled    

### Filtrar el market_Segment ('Direct', 'Corporate', 'Complementary', 'Aviation')

In [168]:
df = df[df['market_segment'].isin(['Direct', 'Corporate',  'Aviation'])]
df.head(3)

      id_reservation        hotel  is_canceled  adults  children  babies meal  \
9787            9787  ResortHotel            1       2       0.0       0   BB   
9792            9792  ResortHotel            1       2       0.0       0   BB   
9799            9799  ResortHotel            1       2       1.0       0   BB   

     country  is_repeated_guest  previous_cancellations  \
9787     BRA                  0                       0   
9792     PRT                  0                       0   
9799     BRA                  0                       0   

      previous_bookings_not_canceled deposit_type market_segment  \
9787                               0   No Deposit         Direct   
9792                               0   No Deposit         Direct   
9799                               0   No Deposit         Direct   

     reservation_status reservation_status_date  arrival_date_year  
9787           Canceled              12/24/2016               2017  
9792           Canceled    

In [169]:
df['market_segment'].unique()

array(['Direct', 'Corporate', 'Aviation'], dtype=object)


### Convertimos todos los valores a minuscula de la columan deposit_type

In [170]:
df['deposit_type'].unique()

array(['No Deposit', 'Non Refund', 'Refundable'], dtype=object)


In [171]:
#Limpiar columna 1. poner a todos minusculas 2. renombrar posibles opciones
df['deposit_type'] = df['deposit_type'].str.lower()

In [172]:
df['deposit_type'].unique()

array(['no deposit', 'non refund', 'refundable'], dtype=object)


### Definimos el tipo de habitación considerando el market_segment y el hotel donde se hospeda

In [173]:
df['market_segment'].unique()

array(['Direct', 'Corporate', 'Aviation'], dtype=object)


In [174]:
df['hotel'].unique()

array(['ResortHotel', 'CityHotel'], dtype=object)


In [3]:
##Crear uan columna tomando como condición otra columna del df

In [175]:
conditions = [
    (df['hotel'] == 'ResortHotel') & (df['market_segment'] == 'Direct'),
    (df['hotel'] == 'ResortHotel') & (df['market_segment'] == 'Corporate'),
    (df['hotel'] == 'ResortHotel') & (df['market_segment'] == 'Aviation'),
    (df['hotel'] == 'CityHotel') & (df['market_segment'] == 'Direct'),
    (df['hotel'] == 'CityHotel') & (df['market_segment'] == 'Aviation') | (df['market_segment'] == 'Corporate')]
choices = ['basic','vip', 'platinium', 'basic' ,'platinium']
df['room_client'] = np.select(conditions, choices, default='to define')


In [176]:
df['room_client'].unique()

array(['basic', 'vip', 'platinium'], dtype=object)


## Guardar datos

In [177]:
df.head(1)

      id_reservation        hotel  is_canceled  adults  children  babies meal  \
9787            9787  ResortHotel            1       2       0.0       0   BB   

     country  is_repeated_guest  previous_cancellations  \
9787     BRA                  0                       0   

      previous_bookings_not_canceled deposit_type market_segment  \
9787                               0   no deposit         Direct   

     reservation_status reservation_status_date  arrival_date_year room_client  
9787           Canceled              12/24/2016               2017       basic


In [140]:
path_target

's3://wo-data-procesada/hotel_data/'


In [ ]:
#Guardar los datos teniendo como partición la columna hotel

In [178]:
wr.s3.to_parquet(
    df=df,
    path=path_target,
    dataset=True,
    partition_cols=['hotel']
)

{'paths': ['s3://wo-data-procesada/hotel_data/hotel=CityHotel/913b731401e949cb987623bb91b3b2ee.snappy.parquet', 's3://wo-data-procesada/hotel_data/hotel=ResortHotel/913b731401e949cb987623bb91b3b2ee.snappy.parquet'], 'partitions_values': {'s3://wo-data-procesada/hotel_data/hotel=CityHotel/': ['CityHotel'], 's3://wo-data-procesada/hotel_data/hotel=ResortHotel/': ['ResortHotel']}}
/home/spark/.local/lib/python3.10/site-packages/awswrangler/s3/_write_dataset.py:153: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for keys, subgroup in df.groupby(by=partition_cols, observed=True):
